In [1]:
import random  
import math
MIN_MUTATION_RATE=0.01
MAX_DISTANCE_SUM=2000
TSP_SIZE=9

# Below are the distances between each of the ten locations that I plan to visit on my trip.
# Location A is the first column and row, location B is the second column and row, etc. For example, 25km is the 
# distance between Location A and Location B, and 43km is the distance between Location A and Location C.
mat = [[ 0, 4.7, 2.4, 7.9, 7.8, 9.4, 5.6, 7.2, 4.0, 5.1], 
       [ 4.7, 0, 1.4, 5.3, 5.2, 5.8, 4.2, 3.2, 3.4, 7.4], 
       [ 2.4, 1.4, 0, 6.2, 5.6, 7.1, 4.4, 4.5, 2.7,6.6], 
       [ 7.9, 5.3, 6.2, 0, 1.3, 3.3, 8.5, 2.4, 7.5,11], 
       [ 7.8, 5.2, 5.6, 1.3, 0, 2, 7.2, 1.7, 7.2, 10], 
       [ 9.4, 5.8, 7.1, 3.3, 2, 0, 8.4, 2.7, 8.9, 11], 
       [ 5.6, 4.2, 4.4, 8.5, 7.2, 8.4, 0, 6.2, 2, 5.7], 
       [ 7.2, 3.2, 4.5, 2.4, 1.7, 2.7, 6.2, 0, 7.3, 9.9], 
       [ 4, 3.4, 2.7, 7.5, 7.2, 8.9, 2, 7.3, 0, 4.5], 
       [ 5.1, 7.4, 6.6, 11, 10, 11, 5.7, 9.9, 4.5, 0]]

# Calculating for the shortest path distance for a 'chromosome' in the genetic algorithm, or a travelling route in 
# this case.
def TSP_distance (chromo):    
    distance=mat[0][chromo[0]]
    loc=0    
    for gene in chromo:
        distance = distance + mat[chromo[loc]][chromo[loc+1]]
        loc=loc+1
        if(loc==len(chromo)-1):
            return(distance)

# As the shorter the route is, the better, we subtract the TSP_distance from MAX_DISTANCE_SUM to attribute a greater
# number to the better (which is the shorter) routes. The fitness function, fitnessftn, is a measure of how short the
# route is.
def fitnessftn (chromo):    
    return(MAX_DISTANCE_SUM-TSP_distance(chromo))

# Generating random travelling routes.
def ran_seq(num): 
    alist=[]
    for i in range(0,num):               
        alist.append(new_rand_ele(num,alist))
    return(alist)

# Generating random numbers (or, locations to visit next) to append to the ran_seq above.
def new_rand_ele(num, alist):
    while 1:
        gennum = int(random.random()*num+1)
        if(ele_in_alist(gennum, alist)==0):
            return(gennum)

def ele_in_alist(num, alist):
    for i in alist:
        if(num==i):
            return 1
    return 0

# Generating an initial population of routes, to then apply the genetic algorithm to.
def init_population(num, size):
    alist=[]
    for i in range (0,size):
        blist=ran_seq(num)
        alist.append(blist)
    return(alist)

# Selecting random routes from the population for 'recombination' to produce 'offsprings'.
# The greater the fitness function, the greater chance a route has of being selected. This is an imitation of the
# survival of the fittest.
def selection(population):
    alist=[]
    totfit=0
    for chromo in population:
        fit=fitnessftn(chromo)
        totfit=totfit+fit
        alist.append(fit)
    randnum=random.random() # generating a random number, randnum
    index=0
    fitsum=0
    for fit in alist:
        fitsum=fitsum+fit       
        if(randnum <fitsum/totfit):
            break
        index=index+1
    return(population[index])
    
# We use a method called uniform order-based crossover to generate 'child' routes from two 'parent' 
# routes to generate a route with hopefully a greater fitness function.
def uniform_order_based_crossover_with_mutation(paren1, paren2, rate):
    template=[]
    resul=[]
    for i in range(0,len(paren1)):
        template.append(int(random.random()*2))
    index=0
    child1=[]
    child2=[]
    rem1=[]
    rem2=[]
    for i in template:
        if(i==1):
            child1.append(paren1[index])
            child2.append(paren2[index])            
        else:
            child1.append(0)
            child2.append(0)
            rem1.append(paren1[index])
            rem2.append(paren2[index])
        index=index+1
    sublist1=sorted_sublist(rem1,paren2)
    sublist2=sorted_sublist(rem2,paren1)
    remindex=0
    index=0
    for i in child1:
        if(i==0):
            child1[index]=sublist1[remindex]
            child2[index]=sublist2[remindex]
            remindex=remindex+1
        index=index+1
    resul.append(per_mutation(child1,rate))
    resul.append(per_mutation(child2,rate))
    return(resul)

def sorted_sublist(small,large):
    alist=[]
    for i in large:
        if(ele_in_alist(i, small)==1):
            alist.append(i)
    return(alist)

def GA_TSP(pop_size,num_gen, mutation_rate, elite_num): # Genetic Algorithm _ The Shortest Path
    # Population size (the number of routes to produce every generation), the number of generatons, the mutation
    # rate, and the number of 'elite chromosomes' to pass on (a certain number of the shortest routes are 
    # automatically passed onto the next generation)
    old_gen=init_population(TSP_SIZE,pop_size)  
    best_route=best_chro(old_gen)  # Finding the best 'chromosome' and attributing it to best_route.
    print("Avg Fit of Gen 0", total_fitness(old_gen)/pop_size)    # Finding the average fitness score of the 
    # first 'generation' of routes generated.
    print("Gen 0 Best Route Fitness",fitnessftn(best_route),best_route) #Finding the best fitness score out of the
    # first 'generation' of routes generated.
    crossoverpairs_num= int((pop_size-elite_num)/2)  
    best_avg_fit=total_fitness(old_gen)/pop_size
    for j in range(0,num_gen):                
        new_gen=[]
        elites=[]
        bbb=elite_chros_indices(old_gen,elite_num)        
        for i in bbb:
            new_gen.append(old_gen[i])
        for i in range(0,crossoverpairs_num):        
            crossovered=uniform_order_based_crossover_with_mutation(selection(old_gen),selection(old_gen),mutation_rate)
            new_gen.extend(crossovered)
        best_route_new_gen=best_chro(new_gen)
        if(fitnessftn(best_route_new_gen)>fitnessftn(best_route)):
            best_route=best_route_new_gen
            print("Gen", j+1,"Best Route Fitness",fitnessftn(best_route),best_route)
        avg_fit=total_fitness(new_gen)/pop_size          
        if(avg_fit>best_avg_fit):
            best_avg_fit=avg_fit
            print("Avg Fit of Gen ", j+1, "=",avg_fit) # print message everytime the best route fitness is improved
            # across generations.
        old_gen=new_gen
    print("Gen", j+1,"Best Route Distance",TSP_distance(best_route),best_route)
    
# Calculating for the total fitness score of a generation of routes.
def total_fitness(population):
    totfit=0
    for chromo in population:
        fit=fitnessftn(chromo)
        totfit=totfit+fit
    return(totfit)

# Finding the best fitness score from a generation of routes.
def best_chro(population):
    MINFIT=-1000
    best_index=0
    index=0    
    best_fit=MINFIT
    for chromo in population:
        fit=fitnessftn(chromo)
        if(best_fit <fit):
            best_index=index
            best_fit=fit
        index=index+1        
    return(population[best_index])

# Finding the index of the best chromosome in a population.
def best_chro_index(alist,exceptlist):
    MINFIT=-1000
    best_index=0
    index=0    
    best_fit=MINFIT
    for chromo in alist:
        if(element_in_list(index,exceptlist)==1):
            index=index+1
            continue
        fit=fitnessftn(chromo)
        if(best_fit <fit):
            best_index=index
            best_fit=fit
        index=index+1        
    return(best_index)

# Collecting the indicies of the best chromosomes, or the shortest routes, produced in a generation.
def elite_chros_indices(alist,num):  
    resul=[]    
    for i in range(0,num):
        best_index=best_chro_index(alist,resul)
        resul.append(best_index)
    return(resul)

def element_in_list(ele,alist):
    for i in alist:
        if(ele==i):
            return(1)
    return(0)

def per_mutation(alist, rate):
    index=0    
    listlen=len(alist)
    firstele=alist[0]
    secondele=alist[1]
    blist=[]
    for i in alist:    
        if(index<listlen-1):
            if(random.random()<rate):
                blist.append(secondele)
            else:
                blist.append(firstele)
                firstele=secondele
        else:
            break
        index=index+1
        if(index<listlen-1):
            secondele=alist[index+1]
        else:
            break
    if(random.random()<rate):
        blist.append(blist[0])        
        blist[0]=firstele
    else:
        blist.append(firstele)        
    return(blist)

In [2]:
ran_seq(5)
# Testing the ran_seq function.

[1, 3, 2, 5, 4]

In [3]:
mat

[[0, 4.7, 2.4, 7.9, 7.8, 9.4, 5.6, 7.2, 4.0, 5.1],
 [4.7, 0, 1.4, 5.3, 5.2, 5.8, 4.2, 3.2, 3.4, 7.4],
 [2.4, 1.4, 0, 6.2, 5.6, 7.1, 4.4, 4.5, 2.7, 6.6],
 [7.9, 5.3, 6.2, 0, 1.3, 3.3, 8.5, 2.4, 7.5, 11],
 [7.8, 5.2, 5.6, 1.3, 0, 2, 7.2, 1.7, 7.2, 10],
 [9.4, 5.8, 7.1, 3.3, 2, 0, 8.4, 2.7, 8.9, 11],
 [5.6, 4.2, 4.4, 8.5, 7.2, 8.4, 0, 6.2, 2, 5.7],
 [7.2, 3.2, 4.5, 2.4, 1.7, 2.7, 6.2, 0, 7.3, 9.9],
 [4, 3.4, 2.7, 7.5, 7.2, 8.9, 2, 7.3, 0, 4.5],
 [5.1, 7.4, 6.6, 11, 10, 11, 5.7, 9.9, 4.5, 0]]

In [4]:
chromosome1 = [1, 2, 3, 4, 5, 6, 7, 8, 9]
chromosome2 = [1, 2, 6, 7, 8, 9, 3, 4, 5]
chromosome3 = [9, 3, 4, 5, 1, 2, 6, 7, 8]
# Test with three random 'chromosomes'. This is how a route would be formatted. 
# For example, chromosome3 is a route that starts from Location 9, then goes to 3, then goes to 4, etc.

In [5]:
print(fitnessftn(chromosome1))
print(fitnessftn(chromosome2))
print(fitnessftn(chromosome3))
# These are the total distances that the travel would take using each of the routes. It can be seen that chromosome3
# is the shortest route.

1958.0
1957.2
1955.5


In [6]:
ipop=init_population(9,40)
# Testing the init_population function to generate a population of routes.

In [7]:
ipop

[[8, 7, 4, 2, 9, 1, 6, 5, 3],
 [5, 3, 2, 7, 1, 4, 8, 9, 6],
 [2, 3, 4, 9, 7, 8, 5, 6, 1],
 [7, 1, 4, 2, 6, 5, 8, 9, 3],
 [5, 7, 8, 9, 4, 1, 6, 2, 3],
 [4, 1, 9, 2, 3, 7, 6, 5, 8],
 [5, 3, 8, 7, 9, 1, 6, 4, 2],
 [6, 4, 2, 1, 5, 9, 3, 8, 7],
 [8, 1, 3, 4, 5, 2, 6, 7, 9],
 [2, 1, 4, 7, 9, 8, 6, 5, 3],
 [7, 5, 4, 3, 1, 8, 6, 2, 9],
 [5, 7, 4, 8, 2, 1, 6, 9, 3],
 [2, 3, 1, 6, 7, 4, 5, 8, 9],
 [2, 3, 6, 1, 7, 9, 4, 8, 5],
 [9, 7, 6, 8, 4, 2, 1, 3, 5],
 [6, 8, 9, 5, 1, 7, 4, 2, 3],
 [1, 4, 8, 6, 7, 2, 3, 9, 5],
 [8, 2, 4, 1, 7, 5, 3, 6, 9],
 [9, 4, 1, 6, 5, 3, 7, 8, 2],
 [9, 5, 7, 8, 2, 4, 6, 1, 3],
 [2, 1, 4, 8, 9, 3, 7, 5, 6],
 [3, 7, 9, 6, 5, 4, 2, 1, 8],
 [8, 3, 7, 9, 4, 1, 2, 6, 5],
 [4, 9, 3, 8, 6, 7, 2, 5, 1],
 [1, 3, 5, 7, 9, 4, 8, 2, 6],
 [4, 9, 3, 5, 2, 1, 7, 8, 6],
 [3, 9, 5, 1, 6, 7, 4, 8, 2],
 [8, 5, 7, 6, 1, 9, 2, 3, 4],
 [2, 7, 8, 5, 1, 6, 4, 3, 9],
 [2, 8, 1, 4, 7, 6, 5, 3, 9],
 [4, 9, 3, 5, 8, 7, 6, 1, 2],
 [5, 4, 1, 6, 7, 2, 3, 9, 8],
 [8, 6, 3, 1, 9, 5, 2, 7, 4],
 [8, 6, 9,

In [8]:
selection(ipop)
# Selecting a random route from ipop.

[2, 1, 4, 8, 9, 3, 7, 5, 6]

In [10]:
GA_TSP(40,1000, 0.02, 0)
# Finding the shortest path distance using 40 routes generated per generation, 1000 generations, and a mutation rate
# of 0.02 (meaning that there is a 2% chance of a mutation occuring).

Avg Fit of Gen 0 1948.225
Gen 0 Best Route Fitness 1961.8 [2, 7, 4, 3, 5, 8, 9, 1, 6]
Avg Fit of Gen  1 = 1948.4325000000003
Gen 2 Best Route Fitness 1963.8 [6, 8, 1, 3, 5, 7, 4, 2, 9]
Avg Fit of Gen  3 = 1950.5900000000001
Gen 4 Best Route Fitness 1966.5 [1, 6, 8, 9, 2, 7, 4, 5, 3]
Gen 6 Best Route Fitness 1967.4 [6, 2, 1, 7, 5, 4, 3, 8, 9]
Avg Fit of Gen  8 = 1951.355
Gen 36 Best Route Fitness 1969.7 [2, 9, 6, 8, 1, 7, 4, 5, 3]
Avg Fit of Gen  40 = 1951.3650000000002
Gen 47 Best Route Fitness 1970.3 [2, 8, 9, 6, 1, 7, 4, 5, 3]
Avg Fit of Gen  47 = 1951.9399999999998
Avg Fit of Gen  48 = 1952.1675
Gen 49 Best Route Fitness 1970.4 [6, 8, 9, 2, 1, 7, 4, 3, 5]
Gen 50 Best Route Fitness 1970.9 [6, 9, 8, 2, 1, 7, 5, 4, 3]
Avg Fit of Gen  50 = 1952.1775000000002
Avg Fit of Gen  51 = 1953.3274999999999
Avg Fit of Gen  58 = 1953.7750000000003
Gen 101 Best Route Fitness 1972.0 [2, 1, 7, 5, 3, 4, 6, 8, 9]
Avg Fit of Gen  210 = 1954.9549999999995
Avg Fit of Gen  211 = 1955.6149999999998
Avg Fit 

In [11]:
# The best route generated is [2, 1, 7, 5, 3, 4, 6, 8, 9] according to the above calculations, and is a distance of 28km.
# If we run the program again:

In [12]:
GA_TSP(40,1000, 0.02, 0)

Avg Fit of Gen 0 1949.6200000000001
Gen 0 Best Route Fitness 1965.6 [2, 8, 9, 6, 7, 3, 5, 4, 1]
Gen 1 Best Route Fitness 1966.8 [9, 6, 8, 3, 5, 4, 7, 2, 1]
Avg Fit of Gen  2 = 1950.595
Gen 10 Best Route Fitness 1967.1 [9, 6, 2, 8, 1, 4, 3, 7, 5]
Avg Fit of Gen  13 = 1951.6200000000001
Gen 14 Best Route Fitness 1967.8 [9, 6, 2, 8, 1, 7, 3, 5, 4]
Avg Fit of Gen  18 = 1952.4874999999993
Gen 39 Best Route Fitness 1968.1 [2, 8, 9, 6, 1, 3, 7, 5, 4]
Gen 69 Best Route Fitness 1969.2 [2, 8, 9, 6, 1, 3, 4, 5, 7]
Gen 87 Best Route Fitness 1970.4 [2, 1, 8, 9, 6, 7, 5, 4, 3]
Avg Fit of Gen  111 = 1952.4975000000009
Gen 112 Best Route Fitness 1970.7 [2, 9, 6, 8, 1, 7, 5, 4, 3]
Avg Fit of Gen  112 = 1953.7899999999997
Avg Fit of Gen  113 = 1954.3925000000004
Gen 115 Best Route Fitness 1970.9 [6, 9, 8, 2, 1, 7, 5, 4, 3]
Avg Fit of Gen  115 = 1954.6125
Gen 116 Best Route Fitness 1973.9 [9, 6, 8, 2, 1, 7, 5, 4, 3]
Avg Fit of Gen  116 = 1955.7350000000006
Gen 1000 Best Route Distance 26.099999999999998 

In [ ]:
# This time, we get the best route as [9, 6, 8, 2, 1, 7, 5, 4, 3], and it has a distance of 26km, which is a bit shorter than
# the previously attained route.
# The genetic algorithm can never guarantee the shortest possible path; that would only be possible by checking
# every single route possible, which would take an astronomical amount of time. The genetic algorithm, however, uses
# a method inspired by evolution in which it constantly survives the routes with higher fitness functions to 
#'reproduce' and pass on their 'desirable genes' (survival of the fittest) to the next 'generation', along with 
# random mutations occuring along the way to make way for the possibility of a randomly generated, but better, route. 